## Non-negative Matrix Factorization

### Import data

In [4]:
import os
import json

dirpath = os.getcwd() + '\\preprocessing\\tokens.json'

with open(dirpath, 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]

In [5]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,март год полный солнечный_затмение полный фаза...
1,сентябрь полный лунный_затмение европейский ча...
2,март год полный солнечный_затмение полный фаза...
3,июль год продолжительный нынешний столетие пол...
4,первый упоминание солнечный_затмение год эра д...


### Build the TF-IDF matrix

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Filter out tokens using TF-IDF
vect = TfidfVectorizer(min_df=.01, max_df=.2)

# Fit and transform
X = vect.fit_transform(df.headline_text)

### Initialize and run the NMF model

In [7]:
from sklearn.decomposition import NMF

# Create an NMF instance
nmf_model = NMF(n_components=20, random_state=5, max_iter=1500)
 
# Fit the model to TF-IDF
nmf_model.fit(X)
 
# Transform the TF-IDF
nmf_features = nmf_model.transform(X)

In [8]:
# Create a dataframe
components_df = pd.DataFrame(nmf_model.components_, columns=vect.get_feature_names())
# components_df

C:\Users\satan\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Get the words of the highest value for each topic

In [9]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:' +
          f'\n{tmp.nlargest(15)}\n')

For topic 1 the words with the highest value are:
губка                      2.197105
личинка                    1.625877
билатерия                  1.342290
позвоночный                1.338457
многоклеточный_животное    1.308273
гребневик                  1.257877
hox                        1.230095
членистоногий              1.195112
ветвь                      1.066213
нервный_система            1.051747
общий_предок               1.036561
двусторонне                0.967506
эволюционный               0.901669
рот                        0.878932
предок                     0.866331
Name: 0, dtype: float64

For topic 2 the words with the highest value are:
самец           2.016292
самка           1.773283
спаривание      0.266713
половой         0.220788
ухаживание      0.217069
паук            0.203466
потомство       0.163199
сперматозоид    0.163019
песня           0.159578
пение           0.159383
особь           0.155369
выбор           0.154454
дрозофила       0.140996
брачный   

### Get the topic of a document

In [10]:
# Look at the nth document
n = 3

# doc = df.headline_text[n]
# print(doc)

pd.DataFrame(nmf_features).loc[n].idxmax() + 1

12

### Get the number of documents for each topic 

In [11]:
pd.DataFrame(nmf_features).idxmax(axis=1).value_counts()

15    149
10    116
5      85
8      79
13     79
14     65
3      64
18     64
7      62
1      62
9      57
4      55
17     55
16     51
2      50
12     43
0      42
19     40
6      27
11     15
dtype: int64